In [1]:
import sys, os, importlib
import rasterio

import numpy as np
import pandas as pd
import geopandas as gpd
import osmnx as ox
import GOSTnets as gn
import skimage.graph as graph

from rasterio.mask import mask
from rasterio import features
from shapely.geometry import box, Point, Polygon
from scipy.ndimage import generic_filter
from pandana.loaders import osm

sys.path.append("../")

import infrasap.market_access as ma
import infrasap.rasterMisc as rMisc
from infrasap.misc import tPrint

# Calculate portsheds

In [2]:
global_friction_surface = "/home/public/Data/GLOBAL/INFRA/FRICTION_2015/2015_friction_surface_v1_no_ocean_travel.tif"

inG = rasterio.open(global_friction_surface)
mcp = graph.MCP_Geometric(inG.read()[0,:,:])

In [3]:
in_ports = "/home/wb411133/data/Projects/INFRA/PORTS/new_major_ports_geocoded.csv"
out_folder = "/home/wb411133/data/Projects/INFRA/PORTS"
ports_file = os.path.join(out_folder, "new_major_ports_20201203.shp")
if not os.path.exists(ports_file):
    inP = pd.read_csv(in_ports)
    inP_geom = [Point(x) for x in zip(inP['Lon'], inP['Lat'])]
    inP = gpd.GeoDataFrame(inP, geometry = inP_geom, crs={'init':'epsg:4326'})
    inP.to_file(ports_file)
else:
    inP = gpd.read_file(ports_file)

In [ ]:
drive_time_thresholds = [0.25, 0.5, 1, 2, 5] # days
drive_time_thresholds = [x * 24 * 60 for x in drive_time_thresholds] #convert days to minutes
drive_vectors = ma.generate_feature_vectors(inG, mcp, inP, drive_time_thresholds)
drive_vectors.to_file(ports_file.replace(".shp", "_PORTSHEDS.shp"))

In [8]:
# Combine datasets together
port_shed_1 = '/home/wb411133/data/Projects/INFRA/PORTS/new_major_ports_20201203_PORTSHEDS.shp'
ports_1 = port_shed_1.replace("_PORTSHEDS","")

port_shed_2 = "/home/wb411133/data/Global/INFRA/PORTS/TRAVEL_TIMES/COMBO_all_ports.shp"
ports_2 = "/home/wb411133/data/Global/INFRA/PORTS/major_ports.shp"

print(f"{os.path.exists(port_shed_1)}: {port_shed_1}")
print(f"{os.path.exists(ports_1)}: {ports_1}")

print(f"{os.path.exists(port_shed_2)}: {port_shed_2}")
print(f"{os.path.exists(ports_2)}: {ports_2}")

True: /home/wb411133/data/Projects/INFRA/PORTS/new_major_ports_20201203_PORTSHEDS.shp
True: /home/wb411133/data/Projects/INFRA/PORTS/new_major_ports_20201203.shp
True: /home/wb411133/data/Global/INFRA/PORTS/TRAVEL_TIMES/COMBO_all_ports.shp
True: /home/wb411133/data/Global/INFRA/PORTS/major_ports.shp


In [95]:
inP2 = gpd.read_file(ports_2)
inP2['Port'] = inP2['Postal'].apply(lambda x: x.split(",")[0])
def get_country(x):
    try:
        return( x.split(",")[1])
    except:
        print(x)
        return("Kenya")

inP2['Country'] = inP2['Postal'].apply(get_country)
inP2 = inP2.drop(['Postal','Lat','Lng','ISO3','OBJECTID'], axis=1)
inP2.columns = [x.upper() for x in inP2.columns]
inP2 = inP2.reindex(sorted(inP2.columns), axis=1)
inP2['IDX'] = inP2.index + 1
inP2.tail()

Lamu Mainland


,COUNTRY,GEOMETRY,PORT,REGION,TYPE,IDX
172,Philippines,POINT (125.4935471693414 9.786834570584471),Surigao,East Asia & Pacific,Bulk,173
173,Philippines,POINT (125.61278 7.073059999999995),Davao,East Asia & Pacific,General,174
174,Papua New Guinea,POINT (147.1585699285962 -9.476931767345329),Port Moresby,East Asia & Pacific,General,175
175,Kenya,POINT (40.91373309719673 -2.199298140720108),Lamu Mainland,Sub-Saharan Africa,General,176
176,Sudan,POINT (37.23221005309645 19.62557566303288),Port Sudan,Sub-Saharan Africa,General,177


In [98]:
inP1 = gpd.read_file(ports_1)
inP1['Type'] = "General"
inP1 = inP1.drop(['Unnamed_ 0','Lat','Lon'],axis=1)
inP1.columns = [x.upper() for x in inP1.columns]
inP1 = inP1.reindex(sorted(inP1.columns), axis=1)
inP1['base'] = inP1.index +1
inP1['IDX'] = inP1.index + max(inP2['IDX'])
inP1.head()

,COUNTRY,GEOMETRY,PORT,REGION,TYPE,base,IDX
0,Indonesia,POINT (128.18333 -3.69583),Ambon,East Asia & Pacific,General,1,177
1,Samoa,POINT (-171.76666 -13.83333),Apia,East Asia & Pacific,General,2,178
2,Thailand,POINT (100.50144 13.75398),Bangkok,East Asia & Pacific,General,3,179
3,Montenegro,POINT (19.09841 42.0937),Bar,Europe & Central Asia,General,4,180
4,Indonesia,POINT (125.12824 1.44059),Bitung,East Asia & Pacific,General,5,181


In [99]:
combo_ports = inP1.append(inP2)

In [126]:
inPortSheds1 = gpd.read_file(port_shed_1)
inPortSheds2 = gpd.read_file(port_shed_2)
inPortSheds2.drop(['IDX'],axis=1,inplace=True)

In [127]:
xx = pd.merge(inPortSheds1, inP1.loc[:,['base','IDX']], left_on='IDX', right_on='base')
xx.drop(['IDX_x','base'],axis=1, inplace=True)
xx.columns = ["threshold","geometry","IDX"]
xx.head()

,threshold,geometry,IDX
0,360.0,POLYGON ((128.2666666666667 -3.508333333333326...,177
1,720.0,POLYGON ((128.2666666666667 -3.508333333333326...,177
2,1440.0,POLYGON ((128.2666666666667 -3.508333333333326...,177
3,2880.0,POLYGON ((128.2666666666667 -3.508333333333326...,177
4,7200.0,POLYGON ((128.2666666666667 -3.508333333333326...,177


In [129]:
inPortSheds2.columns = ['threshold','IDX','geometry']

In [130]:
combo_sheds = inPortSheds2.append(xx)
combo_sheds.head()

,IDX,geometry,threshold
0,12,(POLYGON ((88.93333333333334 25.84166666666667...,360.0
1,12,"(POLYGON ((88.49166666666667 27.4, 88.49166666...",720.0
2,12,"(POLYGON ((91.83333333333331 33.15, 91.8333333...",1440.0
3,12,(POLYGON ((115.3416666666666 27.74166666666667...,2880.0
4,19,(POLYGON ((-37.15000000000001 -4.9416666666666...,360.0


In [132]:
combo_sheds.to_file('/home/wb411133/data/Global/INFRA/PORTS/port_sheds_20201207.shp')

In [137]:
combo_ports.set_geometry("GEOMETRY", inplace=True)

In [138]:
combo_ports.to_file('/home/wb411133/data/Global/INFRA/PORTS/ports_20201207.shp')

# Zonal Stats on VIIRS

In [139]:
port_file = '/home/wb411133/data/Global/INFRA/PORTS/port_sheds_20201207.shp'
zonal_ntl_res = "/home/wb411133/temp/All_Port_VIIRS_ZONAL_Results.csv"
zonal_pop_area_res = "/home/wb411133/temp/All_Port_Pop_Area_Results.csv"
viirs_folder = "/home/public/Data/GLOBAL/NighttimeLights/VIIRS_CORRECTED"
viirs_vrts = []
for root, dirs, files in os.walk(viirs_folder):
    for f in files:
        if f[-4:] == ".vrt":
            viirs_vrts.append(os.path.join(root, f))

inD = inPortSheds #gpd.read_file(port_file)
inD.head()

,IDX,geometry,threshold
0,1,POLYGON ((128.2666666666667 -3.508333333333326...,360.0
1,1,POLYGON ((128.2666666666667 -3.508333333333326...,720.0
2,1,POLYGON ((128.2666666666667 -3.508333333333326...,1440.0
3,1,POLYGON ((128.2666666666667 -3.508333333333326...,2880.0
4,1,POLYGON ((128.2666666666667 -3.508333333333326...,7200.0


In [141]:
if not os.path.exists(zonal_ntl_res):
    all_res = {}
    for vrt in viirs_vrts:
        print(vrt)
        yr = os.path.basename(vrt).split("_")[0]
        res = rMisc.zonalStats(port_file, vrt)
        res = pd.DataFrame(res, columns=["YR_%s_%s" % (yr, x) for x in ['SUM','MIN','MAX','MEAN']])
        all_res[yr] = res

/home/public/Data/GLOBAL/NighttimeLights/VIIRS_CORRECTED/2012_VIIRS_annual_composite.vrt
/home/public/Data/GLOBAL/NighttimeLights/VIIRS_CORRECTED/2013_VIIRS_annual_composite.vrt
/home/public/Data/GLOBAL/NighttimeLights/VIIRS_CORRECTED/2014_VIIRS_annual_composite.vrt
/home/public/Data/GLOBAL/NighttimeLights/VIIRS_CORRECTED/2015_VIIRS_annual_composite.vrt
/home/public/Data/GLOBAL/NighttimeLights/VIIRS_CORRECTED/2016_VIIRS_annual_composite.vrt
/home/public/Data/GLOBAL/NighttimeLights/VIIRS_CORRECTED/2017_VIIRS_annual_composite.vrt
/home/public/Data/GLOBAL/NighttimeLights/VIIRS_CORRECTED/2018_VIIRS_annual_composite.vrt
/home/public/Data/GLOBAL/NighttimeLights/VIIRS_CORRECTED/2019_VIIRS_annual_composite.vrt


ValueError: cannot reindex from a duplicate axis

In [146]:
try:
    del final
except: 
    pass

for key, vals in all_res.items():
    vals['IDX'] = inD['threshold']
    xx = vals.iloc[:,[0,3]]
    try:
        final = final.join(xx)
    except:
        final = xx

In [147]:
final.head()

,YR_2012_SUM,YR_2012_MEAN,YR_2013_SUM,YR_2013_MEAN,YR_2014_SUM,YR_2014_MEAN,YR_2015_SUM,YR_2015_MEAN,YR_2016_SUM,YR_2016_MEAN,YR_2017_SUM,YR_2017_MEAN,YR_2018_SUM,YR_2018_MEAN,YR_2019_SUM,YR_2019_MEAN
0,4.541514e+05,0.715496,4.678778e+05,0.737122,4.773958e+05,0.752117,5.545904e+05,0.873734,4.863387e+05,0.766206,6.556635e+05,1.032970,6.983081e+05,1.100155,6.396908e+05,1.007806
1,1.366994e+06,0.454955,1.476960e+06,0.491553,1.566223e+06,0.521261,1.905424e+06,0.634152,1.750444e+06,0.582572,2.448358e+06,0.814848,2.662940e+06,0.886264,2.494774e+06,0.830296
2,5.782463e+06,0.409109,6.249375e+06,0.442143,6.432250e+06,0.455081,7.176234e+06,0.507718,6.865486e+06,0.485733,9.373895e+06,0.663203,9.829406e+06,0.695430,9.595340e+06,0.678870
3,1.606604e+07,0.303893,1.777559e+07,0.336230,1.798228e+07,0.340140,1.950244e+07,0.368894,1.908534e+07,0.361004,2.518827e+07,0.476443,2.675310e+07,0.506042,2.684634e+07,0.507806
4,5.116798e+05,0.520205,5.835504e+05,0.593273,6.136168e+05,0.623840,6.539396e+05,0.664835,6.668056e+05,0.677915,7.776806e+05,0.790638,8.496504e+05,0.863806,7.933468e+05,0.806565


In [148]:
final['threshold'] = inD['threshold']
final['IDX'] = inD['IDX']
final.to_csv(zonal_ntl_res)

# Zonal stats on Pop

In [ ]:
pop_file = "/home/public/Data/GLOBAL/Population/WorldPop_PPP_2020/ppp_2020_1km_Aggregated.tif"
res = rMisc.zonalStats(inD, pop_file, minVal=0)
res = pd.DataFrame(res, columns=['pop_SUM', 'pop_MIN', 'pop_MAX', 'pop_MEAN'])

In [ ]:
res['threshold'] = inD['threshold']
res['IDX'] = inD['IDX']

# Calculate area

In [ ]:
marketsheds = gpd.read_file(port_file)
marketsheds = marketsheds.to_crs({'init':'epsg:6933'})

marketsheds['AREA_km'] = marketsheds['geometry'].apply(lambda x: x.area/1000000)

In [ ]:
res['AREA_km'] = marketsheds['AREA_km']
res.to_csv(zonal_pop_area_res)

# Combine Results

In [ ]:
inP = gpd.read_file(port_file)


In [ ]:
final['THRESH'] = inP['threshold']
final['PORT'] = inP['PORT']
final['WP_2020_SUM'] = res['pop_SUM']
final['AREA'] = marketsheds['AREA']

In [ ]:
final.to_csv("/home/wb411133/temp/SouthAsia_Port_Results.csv")